# File Format Converter
File format converter is converter module provided by NNabla. It realize Neural Network Libraries (or Console) workflow with ONNX file format, and also NNabla C Runtime. More see [file format converter document](https://nnabla.readthedocs.io/en/latest/python/file_format_converter/file_format_converter.html).

# Functions supported file format converter

* Convert NNP variations to valid NNP

* Convert ONNX to NNP

* Convert NNP to ONNX

* Convert NNP to NNB(Binary format for NNabla C Runtime)

* Convert NNP to Tensorflow saved_model

* Convert Tensorflow checkpoint, frozen graph or saved_model to NNP

* Convert NNP to Tensorflow Lite

* Convert NNP to INT8 quantized Tensorflow Lite

* Convert Tensorflow Lite to NNP

* Experimental: Convert NNP to C Source code for NNabla C Runtime


# How to convert NNP model to ONNX model

In this section, this example shows how to convert Nnabla's NNP model to ONNX model.

####  Install nnabla, nnabla_converter, onnx, matplotlib

In [ ]:
!pip install nnabla nnabla_converter matplotlib onnxruntime

#### Inference by nnabla

In [ ]:
import os
import zlib
import struct
import numpy as np
import nnabla as nn
import matplotlib.pyplot as plt
from nnabla.utils.nnp_graph import NnpLoader
from nnabla.utils.data_source_loader import download
from nnabla.logger import logger

result_dir = "./nnabla_data"
os.makedirs(result_dir, exist_ok=True)


def load_mnist_test_set():
    image_uri = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
    label_uri = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
    logger.info('Getting label data from {}.'.format(label_uri))
    r = download(label_uri, output_file=f"{result_dir}/labels.gz")
    data = zlib.decompress(r.read(), zlib.MAX_WBITS | 32)
    _, size = struct.unpack('>II', data[0:8])
    labels = np.frombuffer(data[8:], np.uint8).reshape(-1, 1)
    r.close()
    logger.info('Getting label data done.')

    logger.info('Getting image data from {}.'.format(image_uri))
    r = download(image_uri, output_file=f"{result_dir}/images.gz")
    data = zlib.decompress(r.read(), zlib.MAX_WBITS | 32)
    _, size, height, width = struct.unpack('>IIII', data[0:16])
    images = np.frombuffer(data[16:], np.uint8).reshape(
        size, 1, height, width)
    r.close()
    logger.info('Getting image data done.')

    return images, labels


def load_nnp(nnp_file):
    
    # download nnp file
    url = "http://nnabla.org/pretrained-models/nnabla-examples/format-converter/mnist.nnp"
    download(url, output_file=nnp_file)
    
    nnp = NnpLoader(nnp_file)
    net = nnp.get_network(nnp.get_network_names()[0], batch_size=1)
    input_var = net.inputs[list(net.inputs.keys())[0]]
    output_var = net.outputs[list(net.outputs.keys())[0]]
    return input_var, output_var


# Test with NNabla
nnp_file = f"{result_dir}/ffc_demo.nnp"
test_imgs, test_labels = load_mnist_test_set()
x, y = load_nnp(nnp_file)
n = 3 # number of images used for demo

for img in test_imgs[:n]:
    input_data = np.expand_dims(img, 0).astype(np.float32) / 255.0
    x.d = input_data
    y.forward()
    pred = np.argmax(y.d[0])
    plt.title('prediction: {}'.format(pred))
    plt.imshow(img[0])
    plt.show()

#### Convert NNabla model to ONNX model

In [ ]:
!nnabla_cli convert -b 1 ./nnabla_data/ffc_demo.nnp ./nnabla_data/ffc_demo.onnx

#### Inference by ONNX

In [ ]:
# Test with ONNX

import onnxruntime as ort

onnxmodel = f"{result_dir}/ffc_demo.onnx"
sessionOptions = ort.SessionOptions()
sessionOptions.intra_op_num_threads = 1
sessionOptions.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
sess = ort.InferenceSession(onnxmodel, sess_options=sessionOptions)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

for img in test_imgs[:n]:
    input_data = np.expand_dims(img, 0).astype(np.float32) / 255.0
    pred = sess.run([output_name], {input_name: input_data})
    pred = np.argmax(pred[0])
    plt.title('prediction: {}'.format(pred))
    plt.imshow(img[0])
    plt.show()

#### Time differences

In [ ]:
# Compare NNabla and ONNX

import time

t1 = time.time()
x.d = input_data
for i in range(1000):
    y.forward()
t2 = time.time()
nnabla_time = t2 - t1

t1 = time.time()
for i in range(1000):
    pred = sess.run([output_name], {input_name: input_data})
t2 = time.time()
onnx_time = t2 - t1

print('NNabla takes {:.3f} seconds to inference 1000 times'.format(nnabla_time))
print('ONNX takes {:.3f} seconds to inference 1000 times'.format(onnx_time))